In [1]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
load_dotenv()

True

In [2]:
openai.organization = os.getenv("OPENAI_ORG_ID")
openai.api_key = os.getenv("OPENAI_API_KEY")

- Models - Language models
- Prompts - Inputs to LM
- Parsers - Parse LM output 


#### Model

In [3]:
chat = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo')

#### Output parser

In [4]:
english_schema = ResponseSchema(
    name='English',
    description='English translation of response'
)

french_schema = ResponseSchema(
    name='French',
    description='French translation of response'
)

response_schema = [english_schema, french_schema]

In [5]:
parser = StructuredOutputParser.from_response_schemas(response_schemas=response_schema)

In [6]:
output_format = parser.get_format_instructions()

#### Prompt template

In [7]:
template = '''
You are an expert in sports, 
your task is to give a brief history of the football club 
delimeted by triple backticks in english and french.

English: 
Frensh:
```{football_club}```

{output_format}
'''

In [9]:
prompt_template = ChatPromptTemplate.from_template(template)

In [10]:
prompt_template.messages[0].prompt.input_variables

['football_club', 'output_format']

#### Query

In [11]:
query = prompt_template.format_messages(
    football_club = 'Barcelona',
    output_format = output_format
)

In [12]:
query[0].content

'\nYou are an expert in sports, \nyour task is to give a brief history of the football club \ndelimeted by triple backticks in english and french.\n\nEnglish: \nFrensh:\n```Barcelona```\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"English": string  // English translation of response\n\t"French": string  // French translation of response\n}\n```\n'

In [13]:
response = chat(query)

In [14]:
response_dict = parser.parse(response.content)

In [15]:
response_dict['English']

"Barcelona Football Club, commonly referred to as Barcelona, is a professional football club based in Barcelona, Catalonia, Spain. Founded in 1899 by a group of Swiss, English, and Catalan footballers led by Joan Gamper, the club has become a symbol of Catalan culture and identity. Barcelona is one of the most successful clubs in Spanish and European football, having won numerous domestic and international titles, including 26 La Liga titles, 31 Copa del Rey titles, and 5 UEFA Champions League titles. The club is known for its attractive style of play, often referred to as 'tiki-taka', and has produced many legendary players such as Johan Cruyff, Lionel Messi, and Xavi Hernandez."

In [16]:
response_dict['French']

"Le Football Club de Barcelone, communément appelé Barcelone, est un club de football professionnel basé à Barcelone, en Catalogne, en Espagne. Fondé en 1899 par un groupe de joueurs de football suisses, anglais et catalans dirigé par Joan Gamper, le club est devenu un symbole de la culture et de l'identité catalanes. Barcelone est l'un des clubs les plus réussis du football espagnol et européen, ayant remporté de nombreux titres nationaux et internationaux, dont 26 titres de La Liga, 31 titres de la Copa del Rey et 5 titres de la Ligue des champions de l'UEFA. Le club est connu pour son style de jeu attrayant, souvent appelé 'tiki-taka', et a produit de nombreux joueurs légendaires tels que Johan Cruyff, Lionel Messi et Xavi Hernandez."

#### Memory management 

In [17]:
from langchain.chains import ConversationChain
from langchain.memory import (ConversationBufferMemory, 
                              ConversationBufferWindowMemory, 
                              ConversationTokenBufferMemory, # token limit
                              ConversationSummaryBufferMemory # summary of conversation with token limit 
                              )

In [18]:
memory = ConversationBufferMemory()

In [19]:
conversation = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True
)

In [ ]:
conversation.predict(input='Tell me about language models')


In [ ]:
conversation.predict(input='why are they not perfect')

In [ ]:
window_memory =ConversationBufferWindowMemory(k=1) # k - Keeps track of n recent messages 

In [ ]:
conversation_windowed = ConversationChain(
    llm=chat,
    memory=window_memory,
    verbose=True
)

In [ ]:
conversation_windowed.predict(input='Hi, my name is folabi')

In [ ]:
conversation_windowed.predict(input='what is my name')

In [ ]:
conversation_windowed.predict(input='what is 1 + 2')

In [ ]:
conversation_windowed.predict(input='what is my name?')

#### Chains 

- LLMChain - model + prompt (sequentially)
- SimpleSequentialChain - runs chains sequentially (Single input variables)
- SequentialChain - runs chains sequentially (multiple input variables)
- Routerchain - combines chains based on a condition


In [20]:
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

In [21]:
llm = ChatOpenAI(
    temperature=0.9,
)

In [22]:
prompt_temp = ChatPromptTemplate.from_template(
    template='Your task is to list five football players that have worn a given jearsy number in barcelona {number}'
)

In [23]:
chain = LLMChain(llm=llm, prompt=prompt_temp)

In [24]:
jearsy_number = 10
chain.run(jearsy_number)

'Five football players who have worn the number 10 jersey for Barcelona are:\n\n1. Ronaldinho: Ronaldinho is widely regarded as one of the greatest players to have worn the number 10 jersey for Barcelona. He played for the club from 2003 to 2008 and was known for his exceptional skill, creativity, and flair on the field.\n\n2. Lionel Messi: After Ronaldinho, Lionel Messi took over the number 10 jersey at Barcelona. Messi played for the club from 2003 to 2021, establishing himself as an all-time great. He won numerous individual awards and led Barcelona to numerous domestic and international titles.\n\n3. Rivaldo: Rivaldo, a Brazilian forward, wore the number 10 jersey for Barcelona from 1997 to 2002. He was a key player for the club during his time, earning the FIFA World Player of the Year award in 1999.\n\n4. Diego Maradona: Although his time at Barcelona was relatively short, Diego Maradona is an iconic figure in football history. He wore the number 10 jersey for the club during the

In [25]:
second_prompt = ChatPromptTemplate.from_template(
    template='Your task is to give a summary about the following players {players}'
)

In [26]:
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [27]:
overall_simple_chain = SimpleSequentialChain(chains=[chain, chain_two],
                                            #  verbose=True
                                            )

In [ ]:
overall_simple_chain.run(jearsy_number)